In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 

In [2]:
class uuCF(object):
    def __init__(self, Y_data, k, sim_func = cosine_similarity):
        self.Y_data = Y_data
        self.k = k # number of neighborhood
        self.sim_func = sim_func 
        self.Ybar = None # normalize data
        self.n_users = int(np.max(self.Y_data[:, 0])) + 1
        self.n_items = int(np.max(self.Y_data[:, 1])) + 1

    def fit(self):
        #normalized
        users = self.Y_data[:, 0] # Lấy tất cả các ID của users
        self.Ybar = self.Y_data.copy() # Sao chép Y_data vào Ybar để thực hiện quá trình chuẩn hóa
        self.mu = np.zeros((self.n_users,)) # Khởi tạo mảng chứa mean rating của mỗi user
        for n in range(self.n_users): # Tìm tất cả các hàng trong Y_data có user_id
            ids = np.where(users == n)[0].astype(int) # Tìm các sản phẩm mà người dùng n đã đánh giá 
            ratings = self.Y_data[ids, 2] # Cột thứ 3 của Y_data - ratings
            self.mu[n] = np.mean(ratings) if ids.size > 0 else 0 # Tính mean rating của người dùng thứ n
            self.Ybar[ids, 2] = ratings - self.mu[n] # Lấy các rating của người dùng n trừ đi mean rating để normalized -> Tránh việc mỗi user có cách đánh giá khác nhau
        self.Ybar = sparse.coo_matrix((self.Ybar[:, 2],                       # Lấy ratings của users
                                      (self.Ybar[:, 1], self.Ybar[:, 0])),    # Lấy item_id -> lấy user_id
                                      (self.n_items, self.n_users)).tocsr()   # Chuyển đổi sparse matrix từ định dạng COO sang CSR
        self.S = self.sim_func(self.Ybar.T, self.Ybar.T) #Tính similarity giữa các người dùng với nhau 

    def pred(self, u, i):
        ids = np.where(self.Y_data[:, 1] == i)[0].astype(int)
        users_rated_i = (self.Y_data[ids, 0]).astype(int)
        sim = self.S[u, users_rated_i]
        nns = np.argsort(sim)[-self.k:]
        nearest_s = sim[nns]
        r = self.Ybar[i, users_rated_i[nns]]
        eps = 1e-8 # a small number to avoid zero division
        return (r*nearest_s).sum()/(np.abs(nearest_s).sum() + eps) + self.mu[u]

In [3]:
#User-User CF
r_cols = ["user_id", "item_id", "rating", "unix_timestamp"]
ratings_base = pd.read_csv("ml-100k/u1.base", sep="\t", names=r_cols)
ratings_test = pd.read_csv("ml-100k/u1.test", sep="\t", names=r_cols)
#Reading ratings file:
xrange = range
rate_train = ratings_base.values
rate_test = ratings_test.values
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1
rs = uuCF(rate_train, k = 15)
rs.fit()
n_tests = rate_test.shape[0]
SE = 0 # squared error
for n in xrange(n_tests):
    pred = rs.pred(rate_test[n, 0], rate_test[n, 1])
    SE += (pred - rate_test[n, 2])**2

RMSE = np.sqrt(SE/n_tests)
print("User-user CF, RMSE =", RMSE)

User-user CF, RMSE = 0.9799732334966004


In [4]:
# def compute_hr_at_10(model, test_data, n_items):
#     hits = 0 # Khởi tạo biến lưu só lần dự đoán đúng nằm trong top 10
#     users = np.unique(test_data[:, 0])  # Lấy tất cả user trong tập test
    
#     for u in users:
#         # Lấy các item thực tế đã được đánh giá bởi user u trong tập test
#         items_tested = test_data[test_data[:, 0] == u][:, 1]
        
#         rated_items = set(model.Y_data[model.Y_data[:, 0] == u][:, 1]) #Lấy danh sách các item đã được người dùng u đánh giá 
#         unrated_items = [i for i in range(n_items) if i not in rated_items] #Lấy danh sách các item chưa được đánh giá bằng cách loại bỏ các item đã được đánh giágiá
 
#         # Dự đoán rating cho các item chưa được đánh giá
#         predictions = [(i, model.pred(u, i)) for i in unrated_items] #Lấy danh sách các dự đoán rating cho các item chưa được đánh giá
#         predictions.sort(key=lambda x: x[1], reverse=True)  # Sắp xếp theo rating dự đoán giảm dần
        
#         # Lấy Top-10 item được dự đoán
#         top_10_items = [item for item, _ in predictions[:10]]
        
#         # Kiểm tra xem có item thực tế nằm trong Top-10 không
#         if any(item in top_10_items for item in items_tested):
#             hits += 1
    
#     hr_at_10 = hits / len(users) # Tính HR@10 = số lần dự đoán đúng / tổng số user
#     return hr_at_10

# hr_at_10 = compute_hr_at_10(rs, rate_test, rs.n_items)
# print("User-user CF, HR@10 =", hr_at_10)

In [5]:
#Item-Item CF
rate_train = rate_train[:, [1, 0, 2]]
rate_test = rate_test[:, [1, 0, 2]]
rs = uuCF(rate_train, k = 40)
rs.fit()
n_tests = rate_test.shape[0]
SE = 0 #square error
for n in xrange(n_tests):
    pred = rs.pred(rate_test[n, 0], rate_test[n, 1])
    SE += (pred - rate_test[n, 2])**2

RMSE = np.sqrt(SE/n_tests)
print("Item-item CF, RMSE =", RMSE)

Item-item CF, RMSE = 0.9657065220682001


In [6]:
def compute_hr_at_10(model, test_data, n_items):
    hits = 0
    users = np.unique(test_data[:, 0])  # Lấy tất cả user trong tập test
    
    for u in tqdm(range(users.shape[0]), desc="Calculating HR@10 with candidates", unit="test"):
        # Lấy các item thực tế đã được đánh giá bởi user u trong tập test
        items_tested = test_data[test_data[:, 0] == u][:, 1] #Lấy danh sách các item đã được người dùng u đánh giá trong tập test
        items_tested = items_tested[np.argsort(items_tested)[::-1][:10]] # Sắp xếp và lấy top 10 ratings cao nhất trong tập item_tested
        # top_10_test_items = items_tested[:10] #Lấy danh sách 10 item đầu tiên trong tập test
        

        rated_items = set(model.Y_data[model.Y_data[:, 0] == u][:, 1]) #Lấy danh sách các item đã được người dùng u đánh giá trong tập huấn luyện
        unrated_items = [i for i in range(n_items) if i not in rated_items] #Lấy danh sách các item chưa được đánh giá bằng cách loại bỏ các item đã được đánh giá ra khỏi n_items
 
        # Dự đoán rating cho các item chưa được đánh giá
        predictions = [(i, model.pred(u, i)) for i in unrated_items] #Lấy danh sách các dự đoán rating cho các item chưa được đánh giá
        predictions.sort(key=lambda x: x[1], reverse=True)  # Sắp xếp theo rating dự đoán giảm dần
        
        # Lấy Top-10 item được dự đoán
        top_10_items = [item for item, _ in predictions[:10]]
        
        # Kiểm tra xem có item thực tế nằm trong Top-10 không
        if any(item in top_10_items for item in items_tested):
            hits += 1
    
    hr_at_10 = hits / len(users) # Tính HR@10 = số lần dự đoán đúng / tổng số user
    return hr_at_10

HR10 = compute_hr_at_10(rs, rate_test, rs.n_items) #Tính HR@10 cho Item-Item CF
print("Item-item CF, HR@10 =", HR10) #In ra kết quả HR@10 cho Item-Item CF

NameError: name 'tqdm' is not defined

In [36]:
def compute_ndcg_at_10(model, test_data, n_items):
    def dcg_at_k(r, k):
        r = np.asfarray(r)[:k]
        if r.size:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        return 0.0

    def ndcg_at_k(r, k):
        dcg_max = dcg_at_k(sorted(r, reverse=True), k)
        if not dcg_max:
            return 0.0
        return dcg_at_k(r, k) / dcg_max

    ndcg_scores = []
    users = np.unique(test_data[:, 0])  # Lấy tất cả user trong tập test
    
    for u in users:
        # Lấy các item thực tế đã được đánh giá bởi user u trong tập test
        items_tested = test_data[test_data[:, 0] == u][:, 1]
        
        # Lấy tất cả item mà user đã đánh giá
        rated_items = set(model.Y_data[model.Y_data[:, 0] == u][:, 1])
        unrated_items = [i for i in range(n_items) if i not in rated_items]

        # Dự đoán rating cho các item chưa được đánh giá
        predictions = [(i, model.pred(u, i)) for i in unrated_items]
        predictions.sort(key=lambda x: x[1], reverse=True)  # Sắp xếp theo rating dự đoán giảm dần
        
        # Lấy Top-10 item được dự đoán
        top_10_items = [item for item, _ in predictions[:10]]
        
        # Tạo danh sách relevance scores cho các item trong top-10
        relevance_scores = [1 if item in items_tested else 0 for item in top_10_items]
        
        # Tính NDCG cho user u
        ndcg = ndcg_at_k(relevance_scores, 10)
        ndcg_scores.append(ndcg)
    
    avg_ndcg_at_10 = np.mean(ndcg_scores)
    return avg_ndcg_at_10

NDCG10 = compute_ndcg_at_10(rs, rate_test, rs.n_items)
print("Item-item CF, NDCG@10 =", NDCG10)

Item-item CF, NDCG@10 = 0.04221968365437769
